# Reddit Survey Assignment
## Evan Thompson


In 2011, Reddit did a survey of it's users. That survey can be found on the class Git repo as RedditShortDemoSurvey-1- Cleaned.csv.
Your assignment is as follows:
1. Clean the survey data and conduct a full EDA. (25 pts)
  - Aggregate all Countries to their Continents.
  - Drop US States
  - Look for and handle missing values
  - Create indicator variables for categoricals, bin where you feel appropriate
  - Clean bad data (e.g. the value movies is present in “Are you a dog or a cat person?”
  - Visualize the distributions of cleaned variables

2. Use Pearson's Correlation Coefficient, determine which variables are most highly collinear, and graph the results. (hint: http://stanford.edu/~mwaskom/ software/seaborn/examples/ many_pairwise_correlations. html) (25pts)

3. Create a random forest model that predicts YOUR dependent variable based on the remaining variables.  If your dependent variable is muti-class (more than just two classes) then I'd like you to use a confusion matrix to measure model performance.  If you'd like, you may also experiment with one-vs-all AUC methods.  (50 pts)

### Determining YOUR dependent variable:
If your Last Name Begins with A-F you will predict Employment
If your Last Name Begins with G-K Marital Status
If your Last Name Begins with L-P Military Service
If your Last Name Begins with Q-Z Income


### I will Predict Income

In [2]:
# With the goal above, I will import just what I need. 
# The model to use (I already imported it above, but will do it again here so each example is self-contained)
from sklearn.ensemble import RandomForestRegressor

# The error metric. In this case, we will use c-stat (aka ROC/AUC)
from sklearn.metrics import roc_auc_score

# An efficient data structure. 
import pandas as pd
import numpy as np
import seaborn as sns


%pylab inline
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)

Python version 2.7.12 |Anaconda 4.0.0 (64-bit)| (default, Jun 29 2016, 11:07:13) [MSC v.1500 64 bit (AMD64)]
Pandas version 0.18.0
Matplotlib version 1.5.1


In [4]:
#import data

X = pd.read_csv("RedditShortDemoSurvey-1-Cleaned.csv")
X.columns  # output the column names

# Standardize Column Names
X.columns = ['entry_id', 'gender', 'categorical_age', 'maritial_status', 'employment_status', 'military_service', 'household_dependents', 'education', 'country', 'us_state', 'annual_gross_income', 'subreddit', 'dog_cat', 'cheese', '14', '15', '16', '17', '18', '19']

# populate dependent variable
y = X.pop("annual_gross_income")

In [5]:
X

,entry_id,gender,categorical_age,maritial_status,employment_status,military_service,household_dependents,education,country,us_state,subreddit,dog_cat,cheese,14,15,16,17,18,19
0,1,Male,25-34,Blank,Employed full time,none,No,Bachelor's degree,United States,New York,getmotivated,none,none,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Male,25-34,Blank,Employed full time,none,No,Bachelor's degree,United States,New York,gaming,none,none,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Female,18-24,Blank,Freelance,none,No,Some college,United States,Virginia,snackexchange,none,none,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Male,25-34,Blank,Freelance,none,No,Bachelor's degree,United States,New York,spacedicks,none,none,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Female,25-34,Blank,Employed full time,none,No,Bachelor's degree,United States,California,aww,none,none,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Male,25-34,Married/civil union/domestic partnership,Employed full time,No,No,Bachelor's degree,United States,New York,gaming,I like dogs.,Cheddar,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Male,25-34,In a relationship,Employed full time,No,No,Some college,Germany,NaN,python,I like dogs.,Cheddar,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Male,18-24,Married/civil union/domestic partnership,"Not employed, but looking for work",No,No,Bachelor's degree,United States,California,fffffffuuuuuuuuuuuu,I like dogs.,American,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Male,35-44,Married/civil union/domestic partnership,Employed full time,No,No,Bachelor's degree,United States,New York,todayilearned,I like cats.,Provolone,NaN,NaN,NaN,NaN,NaN,NaN
9,10,Male,25-34,In a relationship,Freelance,No,No,Some college,Germany,NaN,NaN,I like cats.,Munster,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# read csv file into a list
def get_csv(file1):
    import csv
    
    returnlist = []
    with open(file1, 'U') as csvfile:
        freader = csv.reader(csvfile, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for row in freader:
            returnlist.append(row)
            
    return returnlist

In [7]:
countries_data = get_csv("countries.csv")  # data from https://www.worlddata.info/download/countries.csv

In [8]:
def get_country_dict(country_list):
    #handle data for continent population
    countries = []
    continents = []

    for country in country_list:
        countries.append(country[0].lower())
    
    for continent in country_list:
        continents.append(continent[4].lower())
    
    countries = countries[1:]  #remove header label
    continents = continents[1:]  #remove header label

    country_dict = {}  #dictionary that will hold the country as key and continent as value 

    for entry in range(len(countries)):
        country_dict[countries[entry]] = continents[entry]
        
    return country_dict

In [9]:
country_dict = get_country_dict(countries_data)

In [10]:
df = X

In [11]:
df.describe(include='all')

,entry_id,gender,categorical_age,maritial_status,employment_status,military_service,household_dependents,education,country,us_state,subreddit,dog_cat,cheese,14,15,16,17,18,19
count,32754.000000,32754,32754,32754,32754,32754,32754,32754,32754,20846,28419,32752,32754,3,3,1,1,1,1
unique,NaN,3,8,7,7,3,3,8,440,52,1833,7,17,3,2,1,1,1,1
top,NaN,Male,18-24,Single,Employed full time,No,No,Bachelor's degree,United States,California,askreddit,I like dogs.,Other,7/13/11 15:47,public,I like cats.,Other,7/14/11 13:22,public
freq,NaN,26418,15802,10428,14814,30526,27488,11046,20967,3401,2123,17149,6562,1,2,1,1,1,1
mean,16378.561916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,9456.272427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,8189.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,16379.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,24567.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
continent_list = []

for entry in X['country']:
    cleaned = entry.lower()  #convert entry to lowercase strings
    cleaned = cleaned.strip()  #remove trailing whitespace before and after entry
    cleaned = cleaned.strip('.!,?')
    
    # conditional statements that are formatted (string in cleaned) is searching for a substring in the string to match on
    
    
    if ('ireland' in cleaned):
        continent_list.append(country_dict['ireland'])
    elif (('netherlands' in cleaned) | (cleaned == 'holland')):
        continent_list.append(country_dict['netherlands'])
    elif (('amer' in cleaned) | ('united s' in cleaned) | (cleaned == 'us') | (cleaned == 'usa') | (cleaned == 'us of a')):
        continent_list.append(country_dict['united states'])
    elif ((cleaned == 'peoples republic of south yorkshire') | (cleaned == 'wales')):
        continent_list.append(country_dict['united kingdom'])
    elif (('england' in cleaned) | ('kingdom' in cleaned) | ('britain' in cleaned) | ('britian' in cleaned) | ('united k' in cleaned)):
        continent_list.append(country_dict['united kingdom'])
    elif (('scotland' in cleaned) | (cleaned == "united jingdom") | (cleaned == 'uk')):
        continent_list.append(country_dict['united kingdom'])
    elif ('xico' in cleaned):
        continent_list.append(country_dict['mexico'])
    elif (('korea' in cleaned) & ('south' in cleaned) ):
        continent_list.append(country_dict['south korea'])
    elif ((cleaned == 'cananda') | (cleaned == 'canadia') | (cleaned == 'toronto') | (cleaned == 'canda') | (cleaned == 'camada')):
        continent_list.append(country_dict['canada'])
    elif ('canada' in cleaned):
        continent_list.append(country_dict['canada'])
    elif (('germany' in cleaned) | (cleaned == 'ger')):
        continent_list.append(country_dict['germany'])
    elif ('russia' in cleaned):
        continent_list.append(country_dict['russia'])
    elif ('poland' in cleaned):
        continent_list.append(country_dict['poland'])
    elif ('united a' in cleaned):
        continent_list.append(country_dict['united arab emirates'])
    elif ('moldova' in cleaned):
        continent_list.append(country_dict['moldova'])
    elif (('swe' in cleaned) | ('swd' in cleaned)):
        continent_list.append(country_dict['sweden'])
    elif ('singapore' in cleaned):
        continent_list.append(country_dict['singapore'])
    elif (('china' in cleaned) | (cleaned == 'hong')):
        continent_list.append(country_dict['china'])
    elif (cleaned == 'swiss'):
        continent_list.append(country_dict['switzerland'])
    elif ('fra' in cleaned):
        continent_list.append(country_dict['france'])
    elif (('trinidad' in cleaned) | ('tobago' in cleaned)):
        continent_list.append(country_dict['trinidad and tobago'])
    elif (('holy see' in cleaned)):
        continent_list.append(country_dict['holy see (vatican city)'])
    elif (('austral' in cleaned) | ('stralia' in cleaned)):
        continent_list.append(country_dict['australia'])
    elif (('macedonia' in cleaned)):
        continent_list.append(country_dict['macedonia'])
    elif (('czech' in cleaned)):
        continent_list.append(country_dict['czech republic'])
    elif (('norway' in cleaned)):
        continent_list.append(country_dict['norway'])
    elif (('iraq' in cleaned)):
        continent_list.append(country_dict['iraq'])
    elif ((cleaned == 'brasil')):
        continent_list.append(country_dict['brazil'])
    else:
        try:
            continent_list.append(country_dict[cleaned])
        except KeyError:
            continent_list.append(None)
    
X['continent'] = continent_list

In [13]:
#create copies of the data frames just in case
df_1 = df
X_1 = X

In [14]:
X['has_dependents'] = np.where((X['household_dependents'] == 'Yes') | (X['household_dependents'] ==  'None'), 1, 0)  # create has_dependents from household_dependents to binarize

In [15]:
X['maritial_status'] = np.where((df['maritial_status'] == 'Blank') , 'none', X['maritial_status'])  # convert blanks to none

In [16]:
#collect not working
X['employment_status'] = np.where(((df['employment_status'] == 'Not employed, but looking for work') | (df['employment_status'] == 'Not employed and not looking for work') | (df['employment_status'] == 'None') | (df['employment_status'] == 'Retired')), 'Not Working', X['employment_status'])

In [17]:
X['employment_status'] = np.where((df['employment_status'] == 'Freelance'), 'Employed', X['employment_status']) #convert freelance to employed

In [18]:
X.employment_status.value_counts()

Employed full time    14814
Student               12987
Not Working            3005
Employed               1948
Name: employment_status, dtype: int64

In [19]:
X['dog_cat'] = np.where((df['dog_cat'] == 'I like turtles.'), "None", X['dog_cat'])
X['dog_cat'] = np.where((('dogs' in df['dog_cat'] ) | ('cats' in df['dog_cat'])), X['dog_cat'], "None")

In [20]:
X.cheese.value_counts()

Other            6562
Cheddar          6101
Brie             3742
Provolone        3454
Swiss            3214
Feta             2991
Goat             2516
Munster          1770
American         1438
Paneer            618
Limburger         337
none                5
 f7u12              2
1                   1
programming')       1
 awww               1
 Pics               1
Name: cheese, dtype: int64

In [21]:
X['cheese'] = np.where(((df['cheese'] == 'Pics') | (df['cheese'] == 'f7u12') | (df['cheese'] == 'awww') | (df['cheese'] == '1') | (df['cheese'] == 'programming')), "None", X['cheese'])

In [22]:
X['18-24'] = np.where((df["categorical_age"] == "18-24"), 1, 0)
X['25-34'] = np.where((df["categorical_age"] == "25-34"), 1, 0)
X['under_18'] = np.where((df["categorical_age"] == "Under 18"), 1, 0)
X['35-44'] = np.where((df["categorical_age"] == "35-44"), 1, 0)
X['45-54'] = np.where((df["categorical_age"] == "45-54"), 1, 0)
X['55-64'] = np.where((df["categorical_age"] == "55-64"), 1, 0)
X['over_65'] = np.where((df["categorical_age"] == "Over_65"), 1, 0)

In [23]:
X.describe(include='all')

,entry_id,gender,categorical_age,maritial_status,employment_status,military_service,household_dependents,education,country,us_state,...,19,continent,has_dependents,18-24,25-34,under_18,35-44,45-54,55-64,over_65
count,32754.000000,32754,32754,32754,32754,32754,32754,32754,32754,20846,...,1,32500,32754.000000,32754.000000,32754.000000,32754.000000,32754.000000,32754.000000,32754.000000,32754.0
unique,NaN,3,8,7,4,3,3,8,440,52,...,1,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Male,18-24,Single,Employed full time,No,No,Bachelor's degree,United States,California,...,public,north america,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,26418,15802,10428,14814,30526,27488,11046,20967,3401,...,1,24303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,16378.561916,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.160774,0.482445,0.353392,0.071136,0.068908,0.015326,0.004274,0.0
std,9456.272427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.367328,0.499699,0.478030,0.257056,0.253301,0.122849,0.065239,0.0
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,8189.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,16379.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,24567.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [24]:
X.drop(["14", "15", "16", "17", "18", "19", "entry_id", "us_state", "household_dependents", "categorical_age", "country"], axis=1, inplace=True)

In [25]:
X.describe(include='all')

,gender,maritial_status,employment_status,military_service,education,subreddit,dog_cat,cheese,continent,has_dependents,18-24,25-34,under_18,35-44,45-54,55-64,over_65
count,32754,32754,32754,32754,32754,28419,32754,32754,32500,32754.000000,32754.000000,32754.000000,32754.000000,32754.000000,32754.000000,32754.000000,32754.0
unique,3,7,4,3,8,1833,1,17,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Male,Single,Employed full time,No,Bachelor's degree,askreddit,None,Other,north america,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,26418,10428,14814,30526,11046,2123,32754,6562,24303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.160774,0.482445,0.353392,0.071136,0.068908,0.015326,0.004274,0.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.367328,0.499699,0.478030,0.257056,0.253301,0.122849,0.065239,0.0
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [26]:
X_2 = X

In [27]:
X.describe()

,has_dependents,18-24,25-34,under_18,35-44,45-54,55-64,over_65
count,32754.000000,32754.000000,32754.000000,32754.000000,32754.000000,32754.000000,32754.000000,32754.0
mean,0.160774,0.482445,0.353392,0.071136,0.068908,0.015326,0.004274,0.0
std,0.367328,0.499699,0.478030,0.257056,0.253301,0.122849,0.065239,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0


In [28]:
categorical_variables = ['gender', 'maritial_status', 'employment_status', 'military_service', 'education', 'dog_cat', "subreddit", "cheese", "continent"]

for variable in categorical_variables:
    # Fill missing data with the word "Missing"
    X[variable].fillna("Missing", inplace=True)
    # Create array of dummies
    dummies = pd.get_dummies(X[variable], prefix=variable)
    # Update X to include dummies and drop the main variable
    X = pd.concat([X, dummies], axis=1)
    X.drop([variable], axis=1, inplace=True)

In [29]:
X

,has_dependents,18-24,25-34,under_18,35-44,45-54,55-64,over_65,gender_Female,gender_Male,...,continent_Missing,continent_africa,continent_antarctica,continent_asia,continent_australia,continent_central america,continent_europe,continent_north america,continent_oceania,continent_south america
0,0,0,1,0,0,0,0,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0,0,1,0,0,0,0,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0,1,0,0,0,0,0,0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,0,1,0,0,0,0,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0,0,1,0,0,0,0,0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,0,0,1,0,0,0,0,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0,0,1,0,0,0,0,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,0,1,0,0,0,0,0,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,0,0,0,0,1,0,0,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,0,0,1,0,0,0,0,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [30]:
y = np.where((y == 'Under $20,000'), 19999, y)
y = np.where((y == '$50,000 - $69,999'), 69999, y)
y = np.where((y == '$70,000 - $99,999'), 99999, y)
y = np.where((y == '$100,000 - $149,999'), 149999, y)
y = np.where((y == '$20,000 - $29,999'), 29999, y)
y = np.where((y == '$30,000 - $39,999'), 39999, y)
y = np.where((y == '$150,000 or more'), 150000, y)
y = np.where((y == '$40,000 - $49,999'), 49999, y)
y = np.where((y == 'none'), 0.00, y)

In [31]:
# Make train and test datasets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [36]:
model = RandomForestRegressor(n_estimators=100, oob_score=True, random_state=42)
model.fit(X_train, y_train)
print "R^2:", model.score(X_test, y_test)

R^2: 0.0561384729669


In [37]:
model.oob_score_

0.007877610637335275

In [ ]:
corr = 